In [2]:
!pip install datasets

In [3]:
from datasets import load_dataset
dataset = load_dataset(
    "FinanceInc/auditor_sentiment",
)
print(dataset)

print("訓練集：", dataset['train'])
print("測試集：", dataset['test'])
print("訓練集欄位：", dataset["train"].column_names)
print("測試集欄位：", dataset["test"].column_names)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['sentence', 'label'],
        num_rows: 3877
    })
    test: Dataset({
        features: ['sentence', 'label'],
        num_rows: 969
    })
})
訓練集： Dataset({
    features: ['sentence', 'label'],
    num_rows: 3877
})
測試集： Dataset({
    features: ['sentence', 'label'],
    num_rows: 969
})
訓練集欄位： ['sentence', 'label']
測試集欄位： ['sentence', 'label']


In [4]:
print("\n訓練集前5筆：")
print(dataset["train"][:5])

print("\n測試集前5筆：")
print(dataset["test"][:5])

print("\n資料集的 keys：", dataset.keys())


訓練集前5筆：
{'sentence': ["Altia 's operating profit jumped to EUR 47 million from EUR 6.6 million .", 'The agreement was signed with Biohit Healthcare Ltd , the UK-based subsidiary of Biohit Oyj , a Finnish public company which develops , manufactures and markets liquid handling products and diagnostic test systems .', 'Kesko pursues a strategy of healthy , focused growth concentrating on sales and services to consumer-customers .', 'Vaisala , headquartered in Helsinki in Finland , develops and manufactures electronic measurement systems for meteorology , environmental sciences , traffic and industry .', 'Also , a six-year historic analysis is provided for these markets .'], 'label': [2, 2, 2, 1, 1]}

測試集前5筆：
{'sentence': ["TeliaSonera TLSN said the offer is in line with its strategy to increase its ownership in core business holdings and would strengthen Eesti Telekom 's offering to its customers .", 'STORA ENSO , NORSKE SKOG , M-REAL , UPM-KYMMENE Credit Suisse First Boston ( CFSB ) ra

In [5]:
from collections import Counter
label_counts_train = Counter(dataset["train"]["label"])
print("\n訓練集標籤分佈：", label_counts_train)
label_counts_test = Counter(dataset["test"]["label"])
print("測試集標籤分佈：", label_counts_test)


訓練集標籤分佈： Counter({1: 2320, 2: 1077, 0: 480})
測試集標籤分佈： Counter({1: 559, 2: 286, 0: 124})


In [6]:
import pandas as pd
df_train = pd.DataFrame(dataset["train"])
df_test = pd.DataFrame(dataset["test"])

print("\n訓練集前5筆：")
print(df_train.head())

print("\n測試集前5筆：")
print(df_test.head())


訓練集前5筆：
                                            sentence  label
0  Altia 's operating profit jumped to EUR 47 mil...      2
1  The agreement was signed with Biohit Healthcar...      2
2  Kesko pursues a strategy of healthy , focused ...      2
3  Vaisala , headquartered in Helsinki in Finland...      1
4  Also , a six-year historic analysis is provide...      1

測試集前5筆：
                                            sentence  label
0  TeliaSonera TLSN said the offer is in line wit...      2
1  STORA ENSO , NORSKE SKOG , M-REAL , UPM-KYMMEN...      2
2  Clothing retail chain Sepp+ñl+ñ 's sales incre...      2
3  Lifetree was founded in 2000 , and its revenue...      2
4  Nordea Group 's operating profit increased in ...      2


In [7]:
print("\n訓練集空值：", df_train.isnull().sum())
print("測試集空值：", df_test.isnull().sum())

df_train.drop_duplicates(subset=["sentence"], inplace=True)
df_test.drop_duplicates(subset=["sentence"], inplace=True)

print("\n訓練集大小：", df_train.shape)
print("測試集大小：", df_test.shape)


訓練集空值： sentence    0
label       0
dtype: int64
測試集空值： sentence    0
label       0
dtype: int64

訓練集大小： (3872, 2)
測試集大小： (969, 2)


In [8]:
train_val_split = dataset["train"].train_test_split(test_size=0.2, seed=42)
train_dataset = train_val_split["train"]
val_dataset = train_val_split["test"]
test_dataset = dataset["test"]
print("\n資料集劃分：")
print(f"訓練集大小：{len(train_dataset)}")
print(f"驗證集大小：{len(val_dataset)} （從訓練集切出，用於訓練過程中的評估）")
print(f"測試集大小：{len(test_dataset)} （保留到最後才使用，避免資訊洩漏）")


資料集劃分：
訓練集大小：3101
驗證集大小：776 （從訓練集切出，用於訓練過程中的評估）
測試集大小：969 （保留到最後才使用，避免資訊洩漏）


In [9]:
from transformers import AutoTokenizer
model_name = "roberta-base"
print(f"\n使用的模型：{model_name}")

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(example):
    return tokenizer(example["sentence"], padding="max_length", truncation=True)
tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_val = val_dataset.map(tokenize_function, batched=True)
tokenized_test = test_dataset.map(tokenize_function, batched=True)


使用的模型：roberta-base


Map:   0%|          | 0/776 [00:00<?, ? examples/s]

In [10]:
print("\nTokenized 訓練集第一筆：")
print(tokenized_train[:1])

print("\nTokenized 驗證集第一筆：")
print(tokenized_val[:1])

print("\nTokenized 測試集第一筆：")
print(tokenized_test[:1])

print("\n訓練集特徵：", tokenized_train.features)
print("驗證集特徵：", tokenized_val.features)
print("測試集特徵：", tokenized_test.features)


Tokenized 訓練集第一筆：
{'sentence': ["The financial impact is estimated to be some 1.5 MEUR annual improvement in the division 's result , starting from fiscal year 2007 ."], 'label': [2], 'input_ids': [[0, 133, 613, 913, 16, 2319, 7, 28, 103, 112, 4, 245, 12341, 2492, 1013, 3855, 11, 5, 2757, 128, 29, 898, 2156, 1158, 31, 2358, 76, 3010, 479, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1

In [11]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [12]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)
model.to(device)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
         

In [14]:
from transformers import TrainingArguments
training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    max_steps=50,
    warmup_steps=5,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    report_to="tensorboard",
)

In [15]:
 from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    accuracy = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="weighted")
    return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

In [16]:
from transformers import Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    compute_metrics=compute_metrics,
)
trainer.train()


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
0,0.909200,0.853355,0.601804,0.362168,0.601804,0.452200


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


TrainOutput(global_step=50, training_loss=0.9545834159851074, metrics={'train_runtime': 3670.8442, 'train_samples_per_second': 0.109, 'train_steps_per_second': 0.014, 'total_flos': 105245367091200.0, 'train_loss': 0.9545834159851074, 'epoch': 0.12886597938144329})

In [17]:
print("\n=== 驗證集評估結果 ===")
val_result = trainer.evaluate(eval_dataset=tokenized_val)
print(val_result)


=== 驗證集評估結果 ===


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.8533546328544617, 'eval_accuracy': 0.6018041237113402, 'eval_precision': 0.36216820331597405, 'eval_recall': 0.6018041237113402, 'eval_f1': 0.4522003632714334, 'eval_runtime': 1203.8595, 'eval_samples_per_second': 0.645, 'eval_steps_per_second': 0.081, 'epoch': 0.12886597938144329}


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [18]:
print("\n=== 測試集評估結果（最終評估）===")
test_result = trainer.evaluate(eval_dataset=tokenized_test)
print(test_result)


=== 測試集評估結果（最終評估）===


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.8827455639839172, 'eval_accuracy': 0.5768833849329206, 'eval_precision': 0.3327944398116642, 'eval_recall': 0.5768833849329206, 'eval_f1': 0.42209137719568396, 'eval_runtime': 1499.0882, 'eval_samples_per_second': 0.646, 'eval_steps_per_second': 0.081, 'epoch': 0.12886597938144329}


/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [19]:
print("\n=== 測試集句子展示 ===")
print("以下是測試集中的前 50 個句子：\n")

label_map = {0: "Negative", 1: "Neutral", 2: "Positive"}

for i in range(50):
    sentence = test_dataset[i]["sentence"]
    label = test_dataset[i]["label"]
    print(f"[{i}] {sentence}")
    print(f"    真實標籤：{label_map[label]}\n")


=== 測試集句子展示 ===
以下是測試集中的前 50 個句子：

[0] TeliaSonera TLSN said the offer is in line with its strategy to increase its ownership in core business holdings and would strengthen Eesti Telekom 's offering to its customers .
    真實標籤：Positive

[1] STORA ENSO , NORSKE SKOG , M-REAL , UPM-KYMMENE Credit Suisse First Boston ( CFSB ) raised the fair value for shares in four of the largest Nordic forestry groups .
    真實標籤：Positive

[2] Clothing retail chain Sepp+ñl+ñ 's sales increased by 8 % to EUR 155.2 mn , and operating profit rose to EUR 31.1 mn from EUR 17.1 mn in 2004 .
    真實標籤：Positive

[3] Lifetree was founded in 2000 , and its revenues have risen on an average by 40 % with margins in late 30s .
    真實標籤：Positive

[4] Nordea Group 's operating profit increased in 2010 by 18 percent year-on-year to 3.64 billion euros and total revenue by 3 percent to 9.33 billion euros .
    真實標籤：Positive

[5] Operating profit for the nine-month period increased from EUR3 .1 m and net sales increased fr

In [26]:
test_indices = [20, 23, 25, 29, 32]
print(f"\n你選擇測試的句子索引：{test_indices}")
test_texts = [test_dataset[i]["sentence"] for i in test_indices]
true_labels = [test_dataset[i]["label"] for i in test_indices]

print("\n=== 開始預測 ===\n")
test_encodings = tokenizer(test_texts, truncation=True, padding=True, return_tensors="pt").to(device)
outputs = model(**test_encodings)
preds = torch.argmax(outputs.logits, dim=1).cpu().numpy()
predicted_labels = [label_map[pred] for pred in preds]
true_labels_text = [label_map[label] for label in true_labels]
correct = sum([1 for true, pred in zip(true_labels_text, predicted_labels) if true == pred])
accuracy = correct / len(test_texts) * 100

print(f"你選擇的 {len(test_texts)} 個句子的預測結果：\n")

for i, (idx, text, true_label, pred_label) in enumerate(zip(test_indices, test_texts, true_labels_text, predicted_labels)):
    correct_mark = "✓ 正確" if true_label == pred_label else "✗ 錯誤"
    print(f"{i+1}. 測試集索引 [{idx}]")
    print(f"   句子：{text}")
    print(f"   真實標籤：{true_label}")
    print(f"   預測標籤：{pred_label}")
    print(f"   結果：{correct_mark}\n")

print(f"準確率：{correct}/{len(test_texts)} = {accuracy:.2f}%")


你選擇測試的句子索引：[20, 23, 25, 29, 32]

=== 開始預測 ===

你選擇的 5 個句子的預測結果：

1. 測試集索引 [20]
   句子：In 2009 , Fiskars ' cash flow from operating activities amounted to EUR121m , up from EUR97m in the previous year .
   真實標籤：Positive
   預測標籤：Neutral
   結果：✗ 錯誤

2. 測試集索引 [23]
   句子：The last quarter was the best quarter of 2009 in net sales , and the operating margin rose to 12.2 % .
   真實標籤：Positive
   預測標籤：Neutral
   結果：✗ 錯誤

3. 測試集索引 [25]
   句子：`` The priority for 2009 was to strengthen the company 's balance sheet and increase cash flow , '' CEO Hannu Krook said .
   真實標籤：Positive
   預測標籤：Neutral
   結果：✗ 錯誤

4. 測試集索引 [29]
   句子：Finnish KCI Konecranes has raised its net sales growth estimate for 2006 from over 25 % to over 35 % .
   真實標籤：Positive
   預測標籤：Neutral
   結果：✗ 錯誤

5. 測試集索引 [32]
   句子：Also Lemmink+ñinen 's profit for accounting period went up to EUR 3.1 mn from EUR -24.5 mn a year ago .
   真實標籤：Positive
   預測標籤：Neutral
   結果：✗ 錯誤

準確率：0/5 = 0.00%
